In [1]:
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('bfry100_it4_app').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/13 18:09:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.option("header",True).csv('Data/HeartData2020.csv')
df.printSchema()

root
 |-- Sex: string (nullable = true)
 |-- General_health: string (nullable = true)
 |-- Physical_health_30bad: string (nullable = true)
 |-- Mental_health_30bad: string (nullable = true)
 |-- Medical_cost: string (nullable = true)
 |-- Sleep_hours: string (nullable = true)
 |-- Stroke: string (nullable = true)
 |-- Asthma: string (nullable = true)
 |-- Skin_cancer: string (nullable = true)
 |-- Other_cancer: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Diff_con: string (nullable = true)
 |-- Diff_walk: string (nullable = true)
 |-- Smoked100: string (nullable = true)
 |-- Good_health: string (nullable = true)
 |-- Physical_act: string (nullable = true)
 |-- Heart_disease: string (nullable = true)
 |-- Age_group: string (nullable = true)
 |-- BMI: string (nullable = true)
 |-- Income_group: string (nullable = true)
 |-- Smoking_status: string (nullable = true)
 |-- Heavy_drink: string (nullable = true)



In [3]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType,DoubleType
df2 = df.withColumn("Physical_health_30bad",col("Physical_health_30bad").cast(IntegerType())) \
    .withColumn("Mental_health_30bad",col("Mental_health_30bad").cast(IntegerType())) \
    .withColumn("Sleep_hours",col("Sleep_hours").cast(IntegerType())) \
    .withColumn("BMI",col("BMI").cast(DoubleType()))
df2.printSchema()

root
 |-- Sex: string (nullable = true)
 |-- General_health: string (nullable = true)
 |-- Physical_health_30bad: integer (nullable = true)
 |-- Mental_health_30bad: integer (nullable = true)
 |-- Medical_cost: string (nullable = true)
 |-- Sleep_hours: integer (nullable = true)
 |-- Stroke: string (nullable = true)
 |-- Asthma: string (nullable = true)
 |-- Skin_cancer: string (nullable = true)
 |-- Other_cancer: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Diff_con: string (nullable = true)
 |-- Diff_walk: string (nullable = true)
 |-- Smoked100: string (nullable = true)
 |-- Good_health: string (nullable = true)
 |-- Physical_act: string (nullable = true)
 |-- Heart_disease: string (nullable = true)
 |-- Age_group: string (nullable = true)
 |-- BMI: double (nullable = true)
 |-- Income_group: string (nullable = true)
 |-- Smoking_status: string (nullable = true)
 |-- Heavy_drink: string (nullable = true)



In [4]:
df2.na.drop().count()
df2.count()

273197

In [5]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.util import MLUtils
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import VectorIndexer
from pyspark.mllib.util import MLUtils

cat_features = ["Heart_disease", "Sex", "General_health", "Medical_cost", "Stroke", "Asthma","Skin_cancer", "Other_cancer", "Education", "Diff_con", "Diff_walk", "Smoked100", "Good_health", "Physical_act","Age_group", "Income_group", "Smoking_status","Heavy_drink"]

for i in cat_features:
    indexer = StringIndexer(inputCol=i, outputCol= i+"Index")
    df2 = indexer.fit(df2).transform(df2)
    df2 = df2.drop(i)

In [6]:
df2.printSchema()

root
 |-- Physical_health_30bad: integer (nullable = true)
 |-- Mental_health_30bad: integer (nullable = true)
 |-- Sleep_hours: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- Heart_diseaseIndex: double (nullable = false)
 |-- SexIndex: double (nullable = false)
 |-- General_healthIndex: double (nullable = false)
 |-- Medical_costIndex: double (nullable = false)
 |-- StrokeIndex: double (nullable = false)
 |-- AsthmaIndex: double (nullable = false)
 |-- Skin_cancerIndex: double (nullable = false)
 |-- Other_cancerIndex: double (nullable = false)
 |-- EducationIndex: double (nullable = false)
 |-- Diff_conIndex: double (nullable = false)
 |-- Diff_walkIndex: double (nullable = false)
 |-- Smoked100Index: double (nullable = false)
 |-- Good_healthIndex: double (nullable = false)
 |-- Physical_actIndex: double (nullable = false)
 |-- Age_groupIndex: double (nullable = false)
 |-- Income_groupIndex: double (nullable = false)
 |-- Smoking_statusIndex: double (nullable = f

In [7]:
preds = ["Sex", "General_health", "Medical_cost", "Stroke", "Asthma","Skin_cancer", "Other_cancer", "Education", "Diff_con", "Diff_walk", "Smoked100", "Good_health", "Physical_act","Age_group", "Income_group", "Smoking_status","Heavy_drink"]
numeric = ["Physical_health_30bad", "Mental_health_30bad", "Sleep_hours", "BMI"]
features = []
for i in preds:
    features.append(i+"Index")
for j in numeric:
    features.append(j)

In [8]:
va = VectorAssembler(inputCols = features, outputCol='features')
va_df = va.transform(df2)
va_df = va_df.select(['features', 'Heart_diseaseIndex'])

In [9]:
va_df.show(5)

22/10/13 18:09:42 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+------------------+
|            features|Heart_diseaseIndex|
+--------------------+------------------+
|[0.0,0.0,1.0,0.0,...|               0.0|
|[1.0,3.0,0.0,0.0,...|               0.0|
|(21,[1,5,6,7,8,12...|               0.0|
|(21,[9,13,14,17,1...|               0.0|
|(21,[1,4,5,11,13,...|               0.0|
+--------------------+------------------+
only showing top 5 rows



In [10]:
(train, test) = va_df.randomSplit([0.8, 0.2])

In [11]:
dtc = DecisionTreeClassifier(featuresCol="features", labelCol="Heart_diseaseIndex")
dtc = dtc.fit(train)

In [12]:
feature_importance = dtc.featureImportances

In [13]:
df3 = df2.select([c for c in df2.columns if c in ['Heart_diseaseIndex','SexIndex','General_healthIndex','StrokeIndex', 'Diff_walkIndex', 'Good_healthIndex', 'Age_groupIndex', 'Sleep_hours', 'BMI']])
features = ['SexIndex','General_healthIndex','StrokeIndex', 'Diff_walkIndex', 'Good_healthIndex', 'Age_groupIndex', 'Sleep_hours', 'BMI']
va = VectorAssembler(inputCols = features, outputCol='features')
va_df = va.transform(df3)
va_df = va_df.select(['features', 'Heart_diseaseIndex'])
(train, test) = va_df.randomSplit([0.8, 0.2])



In [15]:
# RandomForestClassifier

from pyspark.ml.classification import RandomForestClassifier


rfc = RandomForestClassifier(featuresCol="features", labelCol="Heart_diseaseIndex")
rfc = rfc.fit(train)

pred = rfc.transform(test)
evaluator=MulticlassClassificationEvaluator(labelCol="Heart_diseaseIndex",predictionCol="prediction")
acc = evaluator.evaluate(pred)
 
print("Prediction Accuracy: ", acc)

Prediction Accuracy:  0.8781700568231042


In [16]:
# GBTClassifier

from pyspark.ml.classification import GBTClassifier

gbc = GBTClassifier(featuresCol="features", labelCol="Heart_diseaseIndex", maxIter=10)
gbc = gbc.fit(train)

pred = dtc.transform(test)
evaluator=MulticlassClassificationEvaluator(labelCol="Heart_diseaseIndex",predictionCol="prediction")
acc = evaluator.evaluate(pred)
 
print("Prediction Accuracy: ", acc)

22/10/13 18:11:05 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/10/13 18:11:05 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


Prediction Accuracy:  0.8841756893575832


In [17]:
# decision tree classifier

dtc = DecisionTreeClassifier(featuresCol="features", labelCol="Heart_diseaseIndex")
dtc = dtc.fit(train)

pred = dtc.transform(test)
evaluator=MulticlassClassificationEvaluator(labelCol="Heart_diseaseIndex",predictionCol="prediction")
acc = evaluator.evaluate(pred)
 
print("Prediction Accuracy: ", acc)

Prediction Accuracy:  0.884985734137341


In [18]:
from sklearn.metrics import confusion_matrix

y_true = pred.select("Heart_diseaseIndex")
y_true = y_true.toPandas()

y_pred = predictions.select("prediction")
y_pred = y_pred.toPandas()

cnf_matrix = confusion_matrix(y_true, y_pred,labels=['yes', 'no'])
cnf_matrix

ModuleNotFoundError: No module named 'sklearn'